In [ ]:

import math
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from IPython.display import display

# Cargar el dataset
df = pd.read_csv('flights_final.csv')

# Función para hacer DFS (Depth First Search)
def dfs(graph, node, visited):
    visited.add(node)
    for neighbor, _ in graph[node]:
        if neighbor not in visited:
            dfs(graph, neighbor, visited)

# Verificar si el grafo es conexo
def is_connected(graph):
    visited = set()
    start_node = list(graph.keys())[0]
    dfs(graph, start_node, visited)
    return len(visited) == len(graph)

# Obtener las componentes conexas
def get_components(graph):
    visited = set()
    components = []

    for node in graph:
        if node not in visited:
            component = set()
            dfs(graph, node, component)
            visited |= component
            components.append(component)

    return components
# Función para mostrar el camino mínimo con información de los aeropuertos intermedios
def show_path_with_airports(graph, start_code, end_code, airports):
    distances, previous_nodes = dijkstra(graph, start_code)

    # Si no hay un camino al destino
    if distances[end_code] == float('inf'):
        print(f"No hay un camino disponible de {start_code} a {end_code}.")
        return

    # Reconstruir el camino desde el destino hasta el origen
    path = []
    current_node = end_code
    while current_node is not None:
        path.append(current_node)
        current_node = previous_nodes[current_node]

    # El camino está al revés, así que lo revertimos
    path.reverse()

    # Mostrar la información del aeropuerto para cada nodo en el camino
    print(f"El camino mínimo desde {start_code} hasta {end_code} es de {distances[end_code]:.2f} km.")
    print("Aeropuertos en el camino:")
    for airport_code in path:
        info = airports.get(airport_code, {})
        name = info.get('name', 'Desconocido')
        city = info.get('city', 'Desconocido')
        country = info.get('country', 'Desconocido')
        latitude = info.get('latitude', 0)
        longitude = info.get('longitude', 0)
        print(f"Código: {airport_code}, Nombre: {name}, Ciudad: {city}, País: {country}, "
              f"Latitud: {latitude}, Longitud: {longitude}")

# Implementación del algoritmo de Kruskal para encontrar el árbol de expansión mínima
def find(parent, i):
    if parent[i] == i:
        return i
    return find(parent, parent[i])

def union(parent, rank, x, y):
    root_x = find(parent, x)
    root_y = find(parent, y)
    if root_x != root_y:
        if rank[root_x] > rank[root_y]:
            parent[root_y] = root_x
        elif rank[root_x] < rank[root_y]:
            parent[root_x] = root_y
        else:
            parent[root_y] = root_x
            rank[root_x] += 1

def kruskal(graph):
    edges = []
    for node in graph:
        for neighbor, weight in graph[node]:
            edges.append((weight, node, neighbor))
    edges.sort()

    parent = {node: node for node in graph}
    rank = {node: 0 for node in graph}

    mst_weight = 0
    mst_edges = []

    for weight, node1, node2 in edges:
        if find(parent, node1) != find(parent, node2):
            union(parent, rank, node1, node2)
            mst_weight += weight
            mst_edges.append((node1, node2, weight))

    return mst_weight, mst_edges

def kruskal_for_components(graph):
    components = get_components(graph)  # Obtener las componentes conexas
    total_mst_weight = 0
    component_msts = []

    for component in components:
        edges = []
        for node in component:
            for neighbor, weight in graph[node]:
                if neighbor in component:  # Solo agregar aristas dentro de la misma componente
                    edges.append((weight, node, neighbor))

        edges.sort()  # Ordenar las aristas por peso

        # Preparar la estructura para Kruskal
        parent = {node: node for node in component}
        rank = {node: 0 for node in component}

        mst_weight = 0
        mst_edges = []

        for weight, node1, node2 in edges:
            if find(parent, node1) != find(parent, node2):
                union(parent, rank, node1, node2)
                mst_weight += weight
                mst_edges.append((node1, node2, weight))

        total_mst_weight += mst_weight
        component_msts.append((mst_weight, mst_edges))

    return total_mst_weight, component_msts

# Algoritmo de Dijkstra para caminos mínimos
def dijkstra(graph, start):
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    previous_nodes = {node: None for node in graph}
    visited = set()

    while len(visited) < len(graph):
        current_node = min((node for node in graph if node not in visited), key=lambda node: distances[node])
        visited.add(current_node)

        for neighbor, weight in graph[current_node]:
            if neighbor in visited:
                continue
            new_distance = distances[current_node] + weight
            if new_distance < distances[neighbor]:
                distances[neighbor] = new_distance
                previous_nodes[neighbor] = current_node

    return distances, previous_nodes

# Mostrar información del aeropuerto
def show_airport_info(airport_code):
    row = df[df['Source Airport Code'] == airport_code].iloc[0]
    info = {
        'Code': row['Source Airport Code'],
        'Name': row['Source Airport Name'],
        'City': row['Source Airport City'],
        'Country': row['Source Airport Country'],
        'Latitude': row['Source Airport Latitude'],
        'Longitude': row['Source Airport Longitude']
    }
    print(f"Información del aeropuerto {airport_code}:")
    for key, value in info.items():
        print(f"{key}: {value}")

# Encontrar los 10 caminos más largos desde un aeropuerto
def find_longest_paths(graph, start, airports):
    distances, _ = dijkstra(graph, start)

    # Filtrar aeropuertos que tengan distancias infinitas
    reachable_airports = {airport: dist for airport, dist in distances.items() if dist != float('inf')}

    # Ordenar los aeropuertos por distancia en orden descendente
    sorted_airports = sorted(reachable_airports.items(), key=lambda x: x[1], reverse=True)

    # Obtener las 10 distancias más largas junto con la información del aeropuerto
    longest_paths_info = []
    for airport, distance in sorted_airports[:10]:  # Limitar a los 10 más lejanos
        info = airports.get(airport, {})
        longest_paths_info.append((airport, distance, info))

    return longest_paths_info

# Función para calcular la distancia entre dos puntos geográficos usando la fórmula de Haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radio de la Tierra en km
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    return R * c

# Crear el grafo a partir de los datos del dataset
graph = {}

for index, row in df.iterrows():
    source = row['Source Airport Code']
    destination = row['Destination Airport Code']
    distance = haversine(row['Source Airport Latitude'], row['Source Airport Longitude'],
                         row['Destination Airport Latitude'], row['Destination Airport Longitude'])

    if source not in graph:
        graph[source] = []
    if destination not in graph:
        graph[destination] = []

    graph[source].append((destination, distance))
    graph[destination].append((source, distance))  # Grafo no dirigido

# Crear un conjunto único de aeropuertos con su información (código, nombre, latitud y longitud)
def get_unique_airports(df):
    airports = {}

    # Para cada fila del dataset, agregamos el aeropuerto de origen y destino al conjunto
    for index, row in df.iterrows():
        source = row['Source Airport Code']
        destination = row['Destination Airport Code']

        # Agregar aeropuerto de origen si no está en el diccionario
        if source not in airports:
            airports[source] = {
                'name': row['Source Airport Name'],
                'city': row['Source Airport City'],
                'country': row['Source Airport Country'],
                'latitude': row['Source Airport Latitude'],
                'longitude': row['Source Airport Longitude']
            }

        # Agregar aeropuerto de destino si no está en el diccionario
        if destination not in airports:
            airports[destination] = {
                'name': row['Destination Airport Name'],
                'city': row['Destination Airport City'],
                'country': row['Destination Airport Country'],
                'latitude': row['Destination Airport Latitude'],
                'longitude': row['Destination Airport Longitude']
            }

    return airports

# Visualización en el mapa usando Folium sin repetir aeropuertos
def create_map(airports):
    m = folium.Map(location=[0, 0], zoom_start=2)
    marker_cluster = MarkerCluster().add_to(m)

    # Añadir marcadores únicos de aeropuertos
    for code, info in airports.items():
        folium.Marker(
            location=[info['latitude'], info['longitude']],
            popup=f"{info['name']} ({code}) - {info['city']}, {info['country']}"
        ).add_to(marker_cluster)

    return m

# Función para mostrar el menú interactivo
def mostrar_menu():
    print("\nMenú de opciones:")
    print("1. Determinar si el grafo es conexo")
    print("2. Determinar el peso del árbol de expansión mínima")
    print("3. Mostrar información de un aeropuerto")
    print("4. Mostrar los 10 aeropuertos más lejanos")
    print("5. Mostrar el camino mínimo entre dos aeropuertos")
    print("6. Mostrar el mapa de aeropuertos")
    print("0. Salir")

def gestionar_menu(graph, airports):
    mostrar_mapa = False
    while True:
        if mostrar_mapa:
            m = create_map(airports)
            display(m)
            input("Presiona Enter para regresar al menú principal")
            mostrar_mapa = False
        else:
            mostrar_menu()
            opcion = input("Seleccione una opción: ")

            if opcion == "1":
                if is_connected(graph):
                    print("El grafo es conexo.")
                else:
                    components = get_components(graph)
                    print(f"El grafo no es conexo. Tiene {len(components)} componentes.")
                    for idx, component in enumerate(components, 1):
                        print(f"Componente {idx}: {len(component)} vértices")
            elif opcion == "2":
                total_weight, component_msts = kruskal_for_components(graph)
                print(f"El peso total de los árboles de expansión mínima es: {total_weight}")
                for idx, (mst_weight, mst_edges) in enumerate(component_msts, 1):
                    print(f"Peso del árbol de expansión mínima de la componente {idx}: {mst_weight}")


            elif opcion == "3":
                code = input("Ingrese el código del aeropuerto: ").upper()
                show_airport_info(code)
            elif opcion == "4":
                code = input("Ingrese el código del aeropuerto: ").upper()
                longest_paths = find_longest_paths(graph, code, airports)
                print("Los 10 aeropuertos más lejanos son:")
                for airport, distance, info in longest_paths:
                  name = info.get('name', 'Desconocido')
                  city = info.get('city', 'Desconocido')
                  country = info.get('country', 'Desconocido')
                  latitude = info.get('latitude', 0)
                  longitude = info.get('longitude', 0)
                  print(f"Aeropuerto {airport} - Nombre: {name} - Ciudad: {city} - País: {country} - Latitud: {latitude} - Longitud: {longitude} - Distancia: {distance:.2f} km")
            elif opcion == "5":
                start_code = input("Ingrese el código del aeropuerto de origen: ").upper()
                end_code = input("Ingrese el código del aeropuerto de destino: ").upper()
                show_path_with_airports(graph, start_code, end_code, airports)
            elif opcion == "6":
                mostrar_mapa = True

            elif opcion == "0":
                print("Saliendo...")
                break
            else:
                print("Opción inválida, por favor intente nuevamente.")

# Llamar al menú de opciones
airports = get_unique_airports(df)
gestionar_menu(graph, airports)



Menú de opciones:
1. Determinar si el grafo es conexo
2. Determinar el peso del árbol de expansión mínima
3. Mostrar información de un aeropuerto
4. Mostrar los 10 aeropuertos más lejanos
5. Mostrar el camino mínimo entre dos aeropuertos
6. Mostrar el mapa de aeropuertos
0. Salir
El grafo no es conexo. Tiene 7 componentes.
Componente 1: 3230 vértices
Componente 2: 4 vértices
Componente 3: 4 vértices
Componente 4: 10 vértices
Componente 5: 4 vértices
Componente 6: 2 vértices
Componente 7: 2 vértices

Menú de opciones:
1. Determinar si el grafo es conexo
2. Determinar el peso del árbol de expansión mínima
3. Mostrar información de un aeropuerto
4. Mostrar los 10 aeropuertos más lejanos
5. Mostrar el camino mínimo entre dos aeropuertos
6. Mostrar el mapa de aeropuertos
0. Salir
Los 10 aeropuertos más lejanos son:
Aeropuerto IPC - Distancia: 19627.05 km
Aeropuerto MPN - Distancia: 19238.10 km
Aeropuerto TBP - Distancia: 18965.66 km
Aeropuerto PIU - Distancia: 18807.59 km
Aeropuerto USH - 